In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gpt-oss:20b", temperature=0)

In [2]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [4]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [5]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt
)

In [6]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [7]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='f5a8280f-789d-48ee-8e35-a43b382d01dd'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2026-02-25T11:16:28.65070441Z', 'done': True, 'done_reason': 'stop', 'total_duration': 236552118256, 'load_duration': 134058557292, 'prompt_eval_count': 268, 'prompt_eval_duration': 81574114669, 'eval_count': 70, 'eval_duration': 19553588546, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019c9480-2d5c-7ac3-bbb2-da41c90e1671-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'langchain-mcp-adapters library'}, 'id': 'f04e190a-2470-411a-9b79-875c2393d528', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 268, 'output_tokens': 70, 'total_tokens': 338}),
              ToolMessage(content=[{'type': 'text', 'text': '{\n  "query": "

In [ ]:
print(response['messages'][-1].content)

## Online MCP

In [9]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [10]:
agent = create_agent(
    model=model,
    tools=tools,
)

In [11]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='f8f09108-256e-43a8-8047-a2b4d247cf81'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-03T20:22:15.514302358Z', 'done': True, 'done_reason': 'stop', 'total_duration': 48337020616, 'load_duration': 205435868, 'prompt_eval_count': 363, 'prompt_eval_duration': 42056111853, 'eval_count': 21, 'eval_duration': 6008899074, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b8585-a5f4-7522-bb02-213ac884ab66-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'America/New_York'}, 'id': '17b731cc-2154-4f43-aa1a-133e720594ae', 'type': 'tool_call'}], usage_metadata={'input_tokens': 363, 'output_tokens': 21, 'total_tokens': 384}),
              ToolMessage(content=[{'type': 'text', 'text': '{\n  "timezone": "America/New_York",\n  "datetime": "2026-01-03T15:22:16-05:00"